# 🧠 Trenowanie i porównanie modeli YOLO w zadaniu detekcji odpadów

## 1. Cel projektu

Celem projektu było:
- wytrenowanie modelu detekcji obiektów typu **YOLO**,
- analiza wpływu jakości danych treningowych oraz liczby epok na jakość modelu,
- porównanie dwóch modeli:
  - **Model A** – trenowany na gorszej jakości zdjęciach i mniejszej liczbie epok,
  - **Model B** – trenowany na lepszej jakości danych i większej liczbie epok.

Projekt dotyczy zadania **detekcji odpadów (Garbage Detection)** na obrazach.

## 2. Opis wykorzystanych modeli

W projekcie wykorzystano architekturę **YOLO11n**, trenowaną w środowisku Python z użyciem biblioteki **Ultralytics YOLO**.

### 2.1 Model A – model słabiej wytrenowany

Model A:
- trenowany na zbiorze danych o **niższej jakości obrazów**,
- trenowany przez **mniejszą liczbę epok**,
- charakteryzuje się:
  - niższym recall,
  - większą liczbą fałszywych detekcji,
  - gorszą lokalizacją obiektów.

### 2.2 Model B – model lepiej wytrenowany

Model B:
- trenowany na **lepszym jakościowo zbiorze danych**,
- trenowany przez **większą liczbę epok**,
- charakteryzuje się:
  - wyższym recall i precision,
  - lepszym dopasowaniem bounding boxów,
  - stabilniejszymi predykcjami.

## 3. Środowisko eksperymentalne

Eksperymenty przeprowadzono w środowisku:
- Python
- Jupyter Notebook
- Ultralytics YOLO
- sprzęt: Apple Silicon (dla modelu A) oraz w Chmurze obliczeniowej (dla modelu B)

Notebook pełni rolę **sprawozdania z eksperymentów** oraz **narzędzia do porównania modeli**.

## 4. Wczytanie bibliotek

In [ ]:
from ultralytics import YOLO

## 5. Wczytanie wytrenowanych modeli

In [ ]:
# Model A – słabiej wytrenowany
model_poor = YOLO("yolo11n run/runs/detect/train/weights/best.onnx")

# Model B – lepiej wytrenowany
model_good = YOLO("GARBAGE CLASSIFICATION 3/runs/detect/train/weights/best.onnx")

## 6. Test modeli na wykonanym przez nas obrazie


In [ ]:
image_path = "/Users/damianbrudkowski/PW/PRiAD/GarbageClassification/resources/glass.jpg"

## 7. Predykcja – Model A (słabiej wytrenowany)

In [ ]:
results_poor = model_poor(image_path)
results_poor[0].show()

Obserwacje:
- model praktycznie nie wykrywa obiektów
- mozliwe błędne detekcje

## 7. Predykcja – Model B (lepiej wytrenowany)

In [ ]:
results_good = model_good(image_path)
results_good[0].show()

Obserwacje:
- model poprawnie wykrywa obiekt
- większa pewność predykcji